In [3]:
import numpy as np
import random
from keras.models import Input,Model
from keras.layers import Dense,Flatten,Reshape
from keras.layers.convolutional import Conv2D
from keras.callbacks import EarlyStopping
from keras.models import load_model
from board_conversion import *
# from chessEngine import *
import chess



In [11]:

class GameState:
    def __init__(self):
        self.board = np.array([
            ["bR", "bN", "bB", "bQ", "bK", "bB", "bN", "bR"],
            ["bp", "bp", "bp", "bp", "bp", "bp", "bp", "bp"],
            ["--", "--", "--", "--", "--", "--", "--", "--"],
            ["--", "--", "--", "--", "--", "--", "--", "--"],
            ["--", "--", "--", "--", "--", "--", "--", "--"],
            ["--", "--", "--", "--", "--", "--", "--", "--"],
            ["wp", "wp", "wp", "wp", "wp", "wp", "wp", "wp"],
            ["wR", "wN", "wB", "wQ", "wK", "wB", "wN", "wR"]
        ])

        self.moveFunctions = {
            'p': self.getPawnMoves,
            'R': self.getRookMoves,
            'N': self.getKnightMoves,
            'B': self.getBishopMoves,
            'Q': self.getQueenMoves,
            'K': self.getKingMoves
        }
        self.whiteToMove = True
        self.moveLog = []
        self.whiteKingLocation = (7, 4)
        self.blackKingLocation = (0, 4)
        self.inCheck = False
        self.staleMate = False
        self.checkMate = False
        self.pins = []
        self.checks = []
        self.enpassantPossible = ()  # coordinates for the square where en passant is possible

        # castling rights
        self.current_castling_rights = CastleRights(True, True, True, True)
        self.castleRightsLog = [
            CastleRights(self.current_castling_rights.wks, self.current_castling_rights.bks,
                         self.current_castling_rights.wqs, self.current_castling_rights.bqs)]

    def makeMove(self, move):
        self.board[move.startRow][move.startCol] = "--"
        self.board[move.endRow][move.endCol] = move.pieceMoved
        self.moveLog.append(move)  # it useful for undo moves
        self.whiteToMove = not self.whiteToMove
        if move.pieceMoved == 'wK':
            self.whiteKingLocation = (move.endRow, move.endCol)
        elif move.pieceMoved == 'bK':
            self.blackKingLocation = (move.endRow, move.endCol)

        # pawn promotion
        if move.pawnPromotion:
            promotedPiece = "Q"
            self.board[move.endRow][move.endCol] = move.pieceMoved[0] + promotedPiece

        # castle moves
        if move.castle:
            if move.endCol - move.startCol == 2:
                self.board[move.endRow][move.endCol - 1] = self.board[move.endRow][move.endCol + 1]  # move rook
                self.board[move.endRow][move.endCol + 1] = '--'
            else:
                self.board[move.endRow][move.endCol + 1] = self.board[move.endRow][move.endCol - 2]
                self.board[move.endRow][move.endCol - 2] = '--'

        # en passant moves
        if move.pieceMoved[1] == 'p' and abs(move.startRow - move.endRow) == 2:
            self.enpassantPossible = ((move.endRow + move.startRow) // 2, move.startCol)
        else:
            self.enpassantPossible = ()
        if move.enPassant:
            self.board[move.startRow][move.startCol] = '--'  # capturing pawn

        # update enpassantPossible variable
        if move.pieceMoved[1] == 'p' and abs(move.startRow - move.endRow) == 2:  # only on 2 square pawn advances
            self.enpassantPossible = ((move.startRow + move.endRow) // 2, move.endCol)
        else:
            self.enpassantPossible = ()

        # if en passant_move, must update the board to capture the pawn
        if move.enPassant:
            self.board[move.startRow][move.endCol] = '--'

        # update castling rights
        self.updateCastleRights(move)
        self.castleRightsLog.append(CastleRights(self.current_castling_rights.wks, self.current_castling_rights.bks,
                                                 self.current_castling_rights.wqs,
                                                 self.current_castling_rights.bqs))

    def undoMove(self):
        if len(self.moveLog) != 0:
            move = self.moveLog.pop()
            self.board[move.startRow][move.startCol] = move.pieceMoved
            self.board[move.endRow][move.endCol] = move.pieceCaptured
            self.whiteToMove = not self.whiteToMove
            if move.pieceMoved == 'wK':
                self.whiteKingLocation = (move.startRow, move.startCol)
            elif move.pieceMoved == 'bK':
                self.blackKingLocation = (move.startRow, move.startCol)

            # undo en passant
            if move.enPassant:
                self.board[move.endRow][move.endCol] = '--'
                self.board[move.startRow][move.endCol] = move.pieceCaptured
                self.enpassantPossible = (move.endRow, move.endCol)

            # undo a 2 square pawn advance
            if move.pieceMoved[1] == 'p' and abs(move.startRow - move.endCol) == 2:
                self.enpassantPossible = ()

            # give back castle rights if move took them away
            self.castleRightsLog.pop()
            self.current_castling_rights = self.castleRightsLog[-1]

            # undo castle
            if move.castle:
                if move.endCol - move.startCol == 2:
                    self.board[move.endRow][move.endCol + 1] = self.board[move.endRow][move.endCol - 1]
                    self.board[move.endRow][move.endCol - 1] = '--'
                else:
                    self.board[move.endRow][move.endCol - 2] = self.board[move.endRow][move.endCol + 1]
                    self.board[move.endRow][move.endCol + 1] = '--'

            self.checkMate = False
            self.staleMate = False

    def getValidMoves(self):
        moves = []
        temp_castle_rights = CastleRights(self.current_castling_rights.wks, self.current_castling_rights.bks,
                                          self.current_castling_rights.wqs, self.current_castling_rights.bqs)
        self.inCheck, self.pins, self.checks = self.checkForPinsAndChecks()
        if self.whiteToMove:
            kingRow = self.whiteKingLocation[0]
            kingCol = self.whiteKingLocation[1]
        else:
            kingRow = self.blackKingLocation[0]
            kingCol = self.blackKingLocation[1]
        if self.inCheck:
            if len(self.checks) == 1:  # only 1 check, block check or move king
                moves = self.getAllPossible()
                # to block a check you must move a piece into one of the square between the enemy piece and king
                check = self.checks[0]
                checkRow = check[0]
                checkCol = check[1]
                pieceChecking = self.board[checkRow][checkCol]
                validSquares = []
                if pieceChecking[1] == 'N':
                    validSquares = [(checkRow, checkCol)]
                else:
                    for i in range(1, 8):
                        validSquare = (
                            kingRow + check[2] * i, kingCol + check[3] * i)  # check[2] and check[3] are check direction
                        validSquares.append(validSquare)
                        if validSquare[0] == checkRow and validSquare[1] == checkCol:
                            break
                # get rid of any moves that dont block check or move king
                for i in range(len(moves) - 1, -1,
                               -1):  # go through baclward when you are removing from a list as iteration
                    if moves[i].pieceMoved[1] != 'K':  # move doesnt move king so it must block or capture
                        if not (moves[i].endRow, moves[i].endCol) in validSquares:
                            moves.remove(moves[i])
            else:  # double check, king has to move
                self.getKingMoves(kingRow, kingCol, moves)
        else:  # not check
            moves = self.getAllPossible()
            if self.whiteToMove:
                self.getCastleMoves(self.whiteKingLocation[0], self.whiteKingLocation[1], moves)
            else:
                self.getCastleMoves(self.blackKingLocation[0], self.blackKingLocation[1], moves)
        if len(moves) == 0:
            if self.inCheck:
                self.checkMate = True
            else:
                self.staleMate = True
        else:
            self.checkMate = False
            self.staleMate = False

        self.current_castling_rights = temp_castle_rights

        return moves


    def checkForPinsAndChecks(self):
        pins = []  # square where the allied pinned piece is and directions pinned from
        checks = []
        inCheck = False
        if self.whiteToMove:
            enemyColor = "b"
            allyColor = "w"
            startRow = self.whiteKingLocation[0]
            startCol = self.whiteKingLocation[1]
        else:
            enemyColor = "w"
            allyColor = "b"
            startRow = self.blackKingLocation[0]
            startCol = self.blackKingLocation[1]

        # check outward from kings for pins and checks, keeps track of pins
        directions = ((-1, 0), (0, -1), (1, 0), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1))
        for j in range(len(directions)):
            d = directions[j]
            possiblePin = ()
            for i in range(1, 8):
                endRow = startRow + d[0] * i
                endCol = startCol + d[1] * i
                if 0 <= endRow < 8 and 0 <= endCol < 8:
                    endPiece = self.board[endRow][endCol]
                    if endPiece[0] == allyColor and endPiece[1] != 'K':
                        if possiblePin == ():
                            possiblePin = (endRow, endCol, d[0], d[1])
                        else:
                            break
                    elif endPiece[0] == enemyColor:
                        type = endPiece[1]
                        if (0 <= j <= 3 and type == 'R') or \
                                (4 <= j <= 7 and type == 'B') or \
                                (i == 1 and type == 'p' and (
                                        (enemyColor == 'w' and 6 <= j <= 7) or (enemyColor == 'b' and 4 <= j <= 5))) or \
                                (type == 'Q') or (i == 1 and type == 'K'):
                            if possiblePin == ():  # no piece blocking, so check
                                inCheck = True
                                checks.append((endRow, endCol, d[0], d[1]))
                                break
                            else:  # piece blocking so pin
                                pins.append(possiblePin)
                                break
                        else:  # enemy piece not applying check
                            break
                else:
                    break

        # check for knight checks
        knightMoves = ((-2, -1), (-2, 1), (-1, -2), (-1, 2), (1, -2), (1, 2), (2, -1), (2, 1))
        for m in knightMoves:
            endRow = startRow + m[0]
            endCol = startCol + m[1]
            if 0 <= endRow < 8 and 0 <= endCol < 8:
                endPiece = self.board[endRow][endCol]
                if endPiece[0] == enemyColor and endPiece[1] == 'N':
                    inCheck = True
                    checks.append((endRow, endCol, m[0], m[1]))

        return inCheck, pins, checks

    def getAllPossible(self):
        moves = []
        for r in range(len(self.board)):
            for c in range(len(self.board[r])):
                turn = self.board[r][c][0]
                if (turn == 'w' and self.whiteToMove) or (turn == 'b' and not self.whiteToMove):
                    piece = self.board[r][c][1]
                    self.moveFunctions[piece](r, c, moves)

        return moves

    def getPawnMoves(self, r, c, moves):
        piecePinned = False
        pinDirection = ()
        for i in range(len(self.pins) - 1, -1, -1):
            if self.pins[i][0] == r and self.pins[i][1] == c:
                piecePinned = True
                pinDirection = (self.pins[i][2], self.pins[i][3])
                self.pins.remove(self.pins[i])
                break

        if self.whiteToMove:
            moveAmount = -1
            startRow = 6
            backRow = 0
            enemyColor = 'b'
        else:
            moveAmount = 1
            startRow = 1
            backRow = 7
            enemyColor = 'w'

        pawnPromotion = False

        if self.board[r + moveAmount][c] == '--':
            if not piecePinned or pinDirection == (moveAmount, 0):
                if r + moveAmount == backRow:
                    pawnPromotion = True
                moves.append(Move((r, c), (r + moveAmount, c), self.board, pawnPromotion=pawnPromotion))
                if r == startRow and self.board[r + 2 * moveAmount][c] == '--':
                    moves.append(Move((r, c), (r + 2 * moveAmount, c), self.board))
        if c - 1 >= 0:
            if not piecePinned or pinDirection == (moveAmount, -1):
                if self.board[r + moveAmount][c - 1][0] == enemyColor:
                    if r + moveAmount == backRow:
                        pawnPromotion = True
                    moves.append(Move((r, c), (r + moveAmount, c - 1), self.board, pawnPromotion=pawnPromotion))
                if (r + moveAmount, c - 1) == self.enpassantPossible:
                    moves.append(Move((r, c), (r + moveAmount, c - 1), self.board, enPassant=True))
        if c + 1 <= 7:
            if not piecePinned or pinDirection == (moveAmount, 1):
                if self.board[r + moveAmount][c + 1][0] == enemyColor:
                    if r + moveAmount == backRow:
                        pawnPromotion = True
                    moves.append(Move((r, c), (r + moveAmount, c + 1), self.board, pawnPromotion=pawnPromotion))
                if (r + moveAmount, c + 1) == self.enpassantPossible:
                    moves.append(Move((r, c), (r + moveAmount, c + 1), self.board, enPassant=True))

    def getRookMoves(self, r, c, moves):
        piecePinned = False
        pinDirection = ()
        for i in range(len(self.pins) - 1, -1, -1):
            if self.pins[i][0] == r and self.pins[i][1] == c:
                piecePinned = True
                pinDirection = (self.pins[i][2], self.pins[i][3])
                if self.board[r][c][1] != 'Q':
                    self.pins.remove(self.pins[i])
                break

        directions = ((-1, 0), (0, -1), (1, 0), (0, 1))
        enemyColor = "b" if self.whiteToMove else "w"
        for d in directions:
            for i in range(1, 8):
                endRow = r + d[0] * i
                endCol = c + d[1] * i
                if 0 <= endRow < 8 and 0 <= endCol < 8:
                    if not piecePinned or pinDirection == d or pinDirection == (-d[0], -d[1]):
                        endPiece = self.board[endRow][endCol]
                        if endPiece == "--":
                            moves.append(Move((r, c), (endRow, endCol), self.board))
                        elif endPiece[0] == enemyColor:
                            moves.append(Move((r, c), (endRow, endCol), self.board))
                            break
                        else:
                            break
                else:
                    break

    def getKnightMoves(self, r, c, moves):
        piecePinned = False
        pinDirection = ()
        for i in range(len(self.pins) - 1, -1, -1):
            if self.pins[i][0] == r and self.pins[i][1] == c:
                piecePinned = True
                pinDirection = (self.pins[i][2], self.pins[i][3])
                self.pins.remove(self.pins[i])
                break
        knightMoves = ((-2, -1), (-2, 1), (-1, -2), (-1, 2), (1, -2), (1, 2), (2, -1), (2, 1))
        allyColor = "w" if self.whiteToMove else "b"
        for m in knightMoves:
            endRow = r + m[0]
            endCol = c + m[1]
            if 0 <= endRow < 8 and 0 <= endCol < 8:
                if not piecePinned:
                    endPiece = self.board[endRow][endCol]
                    if endPiece[0] != allyColor:
                        moves.append(Move((r, c), (endRow, endCol), self.board))

    def getKingMoves(self, r, c, moves):
        rowMoves = (-1, -1, -1, 0, 0, 1, 1, 1)
        colMoves = (-1, 0, 1, -1, 1, -1, 0, 1)
        allyColor = "w" if self.whiteToMove else "b"
        for i in range(8):
            endRow = r + rowMoves[i]
            endCol = c + colMoves[i]
            if 0 <= endRow < 8 and 0 <= endCol < 8:
                endPiece = self.board[endRow][endCol]
                if endPiece[0] != allyColor:
                    if allyColor == 'w':
                        self.whiteKingLocation = (endRow, endCol)
                    else:
                        self.blackKingLocation = (endRow, endCol)
                    inCheck, pins, checks = self.checkForPinsAndChecks()
                    if not inCheck:
                        moves.append(Move((r, c), (endRow, endCol), self.board))
                    if allyColor == 'w':
                        self.whiteKingLocation = (r, c)
                    else:
                        self.blackKingLocation = (r, c)

    def getQueenMoves(self, r, c, moves):
        self.getRookMoves(r, c, moves)
        self.getBishopMoves(r, c, moves)

    def getBishopMoves(self, r, c, moves):
        piecePinned = False
        pinDirection = ()
        for i in range(len(self.pins) - 1, -1, -1):
            if self.pins[i][0] == r and self.pins[i][1] == c:
                piecePinned = True
                pinDirection = (self.pins[i][2], self.pins[i][3])
                self.pins.remove(self.pins[i])
                break

        directions = ((-1, -1), (-1, 1), (1, -1), (1, 1))
        enemyColor = "b" if self.whiteToMove else "w"
        for d in directions:
            for i in range(1, 8):
                endRow = r + d[0] * i
                endCol = c + d[1] * i
                if 0 <= endRow < 8 and 0 <= endCol < 8:
                    if not piecePinned or pinDirection == d or pinDirection == (-d[0], -d[1]):
                        endPiece = self.board[endRow][endCol]
                        if endPiece == "--":
                            moves.append(Move((r, c), (endRow, endCol), self.board))
                        elif endPiece[0] == enemyColor:
                            moves.append(Move((r, c), (endRow, endCol), self.board))
                            break
                        else:
                            break
                else:
                    break

    def updateCastleRights(self, move):
        if move.pieceCaptured == 'wR':
            if move.endCol == 0:
                self.current_castling_rights.wqs = False
            elif move.endCol == 7:
                self.current_castling_rights.wks = False
        elif move.pieceCaptured == 'bR':
            if move.endCol == 0:
                self.current_castling_rights.bqs = False
            elif move.endCol == 7:
                self.current_castling_rights.bks = False

        if move.pieceMoved == 'wk':
            self.current_castling_rights.wks = False
            self.current_castling_rights.wqs = False
        elif move.pieceMoved == 'bk':
            self.current_castling_rights.bks = False
            self.current_castling_rights.bqs = False
        elif move.pieceMoved == 'wR':
            if move.startRow == 7:
                if move.startCol == 7:
                    self.current_castling_rights.wks = False
                elif move.startCol == 0:
                    self.current_castling_rights.wqs = False
        elif move.pieceMoved == 'bR':
            if move.startRow == 0:
                if move.startCol == 7:
                    self.current_castling_rights.bks = False
                elif move.startCol == 0:
                    self.current_castling_rights.bqs = False

    def squareUndarAttack(self, r, c):
        self.whiteToMove = not self.whiteToMove
        opponentMoves = self.getAllPossible()
        self.whiteToMove = not self.whiteToMove
        for move in opponentMoves:
            if r == move.endRow and move.endCol == c:
                return True
        return False

    def getCastleMoves(self, r, c, moves):
        if self.squareUndarAttack(r, c):
            return
        if (self.whiteToMove and self.current_castling_rights.wks) or (
                not self.whiteToMove and self.current_castling_rights.bks):
            self.getKingSideCastleMoves(r, c, moves)
        if (self.whiteToMove and self.current_castling_rights.wqs) or (
                not self.whiteToMove and self.current_castling_rights.bqs):
            self.getQueenSideCastleMoves(r, c, moves)

    def getKingSideCastleMoves(self, r, c, moves):
        if 0 <= r < 8 and 0 <= c + 2 < 8 and 0 <= c + 1 < 8:
            if self.board[r][c + 1] == '--' and self.board[r][c + 2] == '--':
                if not self.squareUndarAttack(r, c + 1) and not self.squareUndarAttack(r, c + 2):
                    moves.append(Move((r, c), (r, c + 2), self.board, castle=True))

    def getQueenSideCastleMoves(self, r, c, moves):
        if 0 <= r < 8 and 0 <= c - 2 < 8 and 0 <= c - 1 < 8 and 0 <= c - 3 < 8 and self.board[r][c - 1] == '--' and \
                self.board[r][c - 2] == '--' and self.board[r][c - 3] == '--' and \
                not self.squareUndarAttack(r,c - 1) and \
                not self.squareUndarAttack(r, c - 2):
            moves.append(Move((r, c), (r, c - 2), self.board, castle=True))


class CastleRights():
    def __init__(self, wks, bks, wqs, bqs):
        self.wks = wks
        self.bks = bks
        self.wqs = wqs
        self.bqs = bqs


class Move:
    # map keys to values
    # key : value
    ranksToRow = {"1": 7, "2": 6, "3": 5, "4": 4,
                  "5": 3, "6": 2, "7": 1, "8": 0}
    rowsToRanks = {v: k for k, v in ranksToRow.items()}
    filesToCols = {"a": 0, "b": 1, "c": 2, "d": 3,
                   "e": 4, "f": 5, "g": 6, "h": 7}
    colsToFiles = {v: k for k, v in filesToCols.items()}

    def __init__(self, startSq, endSq, board, enPassant=False, pawnPromotion=False, castle=False):
        self.startRow = startSq[0]
        self.startCol = startSq[1]
        self.endRow = endSq[0]
        self.endCol = endSq[1]
        self.pieceMoved = board[self.startRow][self.startCol]
        self.pieceCaptured = board[self.endRow][self.endCol]
        self.moveID = self.startRow * 1000 + self.startCol * 100 + self.endRow * 10 + self.endCol
        self.pawnPromotion = pawnPromotion
        self.castle = castle
        self.enPassant = enPassant
        if enPassant:
            self.pieceCaptured = 'wp' if self.pieceMoved == 'bp' else 'bp'

    # Overriding equal method
    def __eq__(self, other):
        if isinstance(other, Move):
            return self.moveID == other.moveID
        return False

    def getRankFiles(self, r, c):
        return self.colsToFiles[c] + self.rowsToRanks[r]

    def getChessNotation(self):
        return self.getRankFiles(self.startRow, self.startCol) + self.getRankFiles(self.endRow, self.endCol)


In [4]:

class NeuralNetwork():
    def __init__(self):
        self.optimizer = 'Adam'
        self.loss = 'categorical_crossentropy'

    def define(self):
        input_layer = Input(shape=(8, 8, 12))
        x = Conv2D(filters=64, kernel_size=2, strides=(2, 2))(input_layer)
        x = Conv2D(filters=128, kernel_size=2, strides=(2, 2))(x)
        x = Conv2D(filters=256, kernel_size=2, strides=(2, 2))(x)
        x = Flatten()(x)

        x = Dense(4096, activation='softmax')(x)
        output = Reshape((1, 64, 64))(x)

        model = Model(inputs=input_layer, outputs=output)
        model.compile(optimizer=self.optimzier, loss=self.loss)
        self.model = model

    def train(self, X, y, epochs, EarlyStop=True):
        if EarlyStop:
            es = EarlyStopping(monitor='loss')

        self.model.fit(X, y, epochs=epochs, callbacks=[es])
        self.model.save('chess_model')

    def predict(self, board, side):
        model = load_model("chess_model")
        translated = translate_board(board)

        move_matrix = model(translated.reshape(1, 8, 8, 12))[0][0]
        move_matrix = filter_legal_moves(board, move_matrix)
        move = np.unravel_index(np.argmax(move_matrix, axis=None), move_matrix.shape)
        move = chess.Move(move[0], move[1])
        return move, 1


In [5]:
pieceScore = {"K": 0, "Q": 9, "R": 5, "N": 3, "B": 3, "p": 1}

knight_score = np.array([
    [0.0, 0.1, 0.2, 0.2, 0.2, 0.2, 0.1, 0.0],
    [0.1, 0.3, 0.5, 0.5, 0.5, 0.5, 0.3, 0.1],
    [0.2, 0.5, 0.6, 0.65, 0.65, 0.6, 0.5, 0.2],
    [0.2, 0.55, 0.65, 0.7, 0.7, 0.65, 0.55, 0.2],
    [0.2, 0.5, 0.65, 0.7, 0.7, 0.65, 0.5, 0.2],
    [0.2, 0.55, 0.6, 0.65, 0.65, 0.6, 0.55, 0.2],
    [0.1, 0.3, 0.5, 0.55, 0.55, 0.5, 0.3, 0.1],
    [0.0, 0.1, 0.2, 0.2, 0.2, 0.2, 0.1, 0.0]
])

bishop_score = np.array([
    [0.0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.0],
    [0.2, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.2],
    [0.2, 0.4, 0.5, 0.6, 0.6, 0.5, 0.4, 0.2],
    [0.2, 0.5, 0.5, 0.6, 0.6, 0.5, 0.5, 0.2],
    [0.2, 0.4, 0.6, 0.6, 0.6, 0.6, 0.4, 0.2],
    [0.2, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.2],
    [0.2, 0.5, 0.4, 0.4, 0.4, 0.4, 0.5, 0.2],
    [0.0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.0]
])

rook_score = np.array([
    [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
    [0.5, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.5],
    [0.0, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.0],
    [0.0, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.0],
    [0.0, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.0],
    [0.0, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.0],
    [0.0, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.0],
    [0.25, 0.25, 0.25, 0.5, 0.5, 0.25, 0.25, 0.25]
])

queen_score = np.array([
    [0.0, 0.2, 0.2, 0.3, 0.3, 0.2, 0.2, 0.0],
    [0.2, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.2],
    [0.2, 0.4, 0.5, 0.5, 0.5, 0.5, 0.4, 0.2],
    [0.3, 0.4, 0.5, 0.5, 0.5, 0.5, 0.4, 0.3],
    [0.4, 0.4, 0.5, 0.5, 0.5, 0.5, 0.4, 0.3],
    [0.2, 0.5, 0.5, 0.5, 0.5, 0.5, 0.4, 0.2],
    [0.2, 0.4, 0.5, 0.4, 0.4, 0.4, 0.4, 0.2],
    [0.0, 0.2, 0.2, 0.3, 0.3, 0.2, 0.2, 0.0]
])

pawn_score = np.array([
    [0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8],
    [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7],
    [0.3, 0.3, 0.4, 0.5, 0.5, 0.4, 0.3, 0.3],
    [0.25, 0.25, 0.3, 0.45, 0.45, 0.3, 0.25, 0.25],
    [0.2, 0.2, 0.2, 0.4, 0.4, 0.2, 0.2, 0.2],
    [0.25, 0.15, 0.1, 0.2, 0.2, 0.1, 0.15, 0.25],
    [0.25, 0.3, 0.3, 0.0, 0.0, 0.3, 0.3, 0.25],
    [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
])

piece_position_scores = {
    "wN": knight_score,
    "bN": knight_score[::-1],
    "wB": bishop_score,
    "bB": bishop_score[::-1],
    "wQ": queen_score,
    "bQ": queen_score[::-1],
    "wR": rook_score,
    "bR": rook_score[::-1],
    "wp": pawn_score,
    "bp": pawn_score[::-1],
}

CHECKMATE = 1000
STALEMATE = 0
DEPTH = 1


In [6]:


def scoreMaterial(game_state):
    if game_state.checkMate:
        if game_state.whiteToMove:
            return -CHECKMATE  # black win
        else:
            return CHECKMATE  # white win
    elif game_state.staleMate:
        return STALEMATE
    score = 0
    for r in range(len(game_state.board)):
        for c in range(len(game_state.board[r])):
            piece = game_state.board[r][c]
            if piece != "--":
                piece_position_score = 0
                if piece[1] != "K":
                    piece_position_score = piece_position_scores[piece][r][c]
                if piece[0] == "w":
                    score += pieceScore[piece[1]] + piece_position_score
                if piece[0] == "b":
                    score -= pieceScore[piece[1]] + piece_position_score
    return score

In [20]:
def boardToFen(gs,board):
    a = np.char.replace(board, "bR", "r")
    a = np.char.replace(a, "bN", "n")
    a = np.char.replace(a, "bB", "b")
    a = np.char.replace(a, "bQ", "q")
    a = np.char.replace(a, "bK", "k")
    a = np.char.replace(a, "bp", "p")

    a = np.char.replace(a, "wN", "N")
    a = np.char.replace(a, "wB", "B")
    a = np.char.replace(a, "wQ", "Q")
    a = np.char.replace(a, "wK", "K")
    a = np.char.replace(a, "wp", "P")
    a = np.char.replace(a, "wR", "R")
    a = np.char.replace(a, "--", "")

    fen = ""
    RANK_SEPERATOR = "/"
    for rank in range(8):
        empty = 0
        rankFen = ""
        for file in range(8):
            if len(a[rank][file]) == 0:
                empty = empty + 1
            else:
                if empty != 0:
                    rankFen += str(empty)
                rankFen += a[rank][file]
                empty = 0
        if empty != 0:
            rankFen += str(empty)
        fen += rankFen
        if not (rank == len(board) - 1):
            fen += RANK_SEPERATOR
        else:
            fen += " "
    side = 'w' if gs.whiteToMove else 'b'
    fen += ' ' + side
    return fen


In [29]:

def findAlphaBeta(gs, valid_moves, depth, alpha, beta, turn_multiplayer):
    global bestMove
    side = 'White' if gs.whiteToMove else 'Black'
    if depth == 0:
        board = chess.Board(boardToFen(gs,gs.board))
        engine = NeuralNetwork()
        flag = False
        move =  engine.predict(board,side)
        for i in range(len(validMoves)):
            if str(validMoves[i].getChessNotation()) == str(move):
                gs.makeMove(validMoves[i])
                flag = True
                break
        t = turn_multiplayer * scoreMaterial(gs)
        if flag:
            gs.undoMove()
        return t
    max_score = -CHECKMATE
    for move in valid_moves:
        gs.makeMove(move)
        bestMoves = gs.getValidMoves()
        score = -findAlphaBeta(gs, bestMoves, depth - 1, -beta, -alpha, -turn_multiplayer)
        if score > max_score:
            max_score = score
            if depth == DEPTH:
                bestMove = move
        gs.undoMove()
        if max_score > alpha:
            alpha = max_score
        if alpha >= beta:
            break
    return max_score


def findBestMoves(gs, validmoves):
    global bestMove
    bestMove = None
    random.shuffle(validmoves)
    s = findAlphaBeta(gs, validmoves, DEPTH, -CHECKMATE, CHECKMATE, 1 if gs.whiteToMove else -1)
#    print(bestMove.moveID)
    return bestMove


In [30]:
gs = GameState()
side = 'White' if gs.whiteToMove else 'Black'

In [31]:
print(gs.board)

[['bR' 'bN' 'bB' 'bQ' 'bK' 'bB' 'bN' 'bR']
 ['bp' 'bp' 'bp' 'bp' 'bp' 'bp' 'bp' 'bp']
 ['--' '--' '--' '--' '--' '--' '--' '--']
 ['--' '--' '--' '--' '--' '--' '--' '--']
 ['--' '--' '--' '--' '--' '--' '--' '--']
 ['--' '--' '--' '--' '--' '--' '--' '--']
 ['wp' 'wp' 'wp' 'wp' 'wp' 'wp' 'wp' 'wp']
 ['wR' 'wN' 'wB' 'wQ' 'wK' 'wB' 'wN' 'wR']]


In [32]:
validMoves = gs.getValidMoves()
move = findBestMoves(gs,validMoves)
print(move)

In [33]:
print(move.getChessNotation())
gs.makeMove(move)
print(gs.board)

g1f3
[['bR' 'bN' 'bB' 'bQ' 'bK' 'bB' 'bN' 'bR']
 ['bp' 'bp' 'bp' 'bp' 'bp' 'bp' 'bp' 'bp']
 ['--' '--' '--' '--' '--' '--' '--' '--']
 ['--' '--' '--' '--' '--' '--' '--' '--']
 ['--' '--' '--' '--' '--' '--' '--' '--']
 ['--' '--' '--' '--' '--' 'wN' '--' '--']
 ['wp' 'wp' 'wp' 'wp' 'wp' 'wp' 'wp' 'wp']
 ['wR' 'wN' 'wB' 'wQ' 'wK' 'wB' '--' 'wR']]


In [34]:
side = 'White' if gs.whiteToMove else 'Black'
validMoves = gs.getValidMoves()
move = findBestMoves(gs,validMoves)